In [1]:
import numpy as np
import os
import glob
import cv2
from scipy.spatial.distance import cdist
import random
from PIL import Image

# K-means

Aim to find an assignment of data points to clusters, As well as a set of vectors $\{\mu_{k}\}$, such that the sum of squares of the distances of each data points to its closest vector $\mu_{k}$ is a minimum. 

## Load Image

***info_C means the RGB information of images.(RGB value)***<br>
C : (pixel, (B, G, R))<br>
***info_S means the spatial information of images.(coordinate of pixel)***<br>
S : (pixel, (x, y))<br>

In [2]:
def extract_info(image):
    img_C = image.reshape((10000,3)) 
    img_S = np.array([(i,j) for i in range(100) for j in range(100)])
    return img_S, img_C

## Kernel

From the lecture ML Week3 Part1, page 23 in ppt: <br>
By using the reorder data to compute the ***Gram matrix***.

$k(x,x') = e^{-\gamma_{s}||S(x)-S(x')||^{2}} \times e^{-\gamma_{c}||C(x)-C(x')||^{2}}$

In [3]:
def kernel(gamma_S, gamma_C, S, C):
    result_S = np.exp(-gamma_S*cdist(S, S, 'sqeuclidean'))
    result_C = np.exp(-gamma_C*cdist(C, C, 'sqeuclidean'))
    Gram_matrix = np.multiply(result_S, result_C)
    return Gram_matrix

## Initial

Initialize centers $\mu_{k}$ (e.g. randomly pick $k$ data points as centers)<br>

In [4]:
def initial_center(K, mode):
    #randomly pick k data points as centers
    center_idx = []
    if mode == 'random':
        center_idx = list(random.sample(range(0, 10000), K))
    else: 
        random_idx = random.sample(range(0, 10000), 1)
        
        for r in range(K):
            dist = np.zeros(10000)
            for i in range(10000):
                min_dist = np.inf
                cur_dist = np.linalg.norm(info_S[i]-info_S[random_idx])
                if cur_dist < min_dist:
                    min_dist = cur_dist
                dist[i] = min_dist
            pob = dist/np.sum(dist)
            idx = np.random.choice(np.arange(10000), 1, p=pob)
            center_idx.append(idx[0])
    
    return center_idx

You can first apply an initial clustering, then use its result to reorder your data.<br>


In [5]:
def initial_cluster(center, phi_X):
    cluster = np.zeros(len(phi_X), dtype=int)
    for n in range(len(phi_X)):
        distance = np.zeros(len(center))
        for c in range(len(center)):
            distance[c] = phi_X[n, n] + phi_X[center[c], center[c]] -2*phi_X[n, center[c]]
        cluster[n] = np.argmin(distance)
    return cluster

## Distance formula in kernel space

$||\phi(x_{j})-\mu_{k}^{\phi}|| = \phi(x_{j})-\frac{1}{|C_{k}|}\sum_{n=1}^{N}\alpha_{kn}\phi(x_{n})||\\
= k(x_{j},x_{j})-\frac{2}{|C_{k}|}\sum_{n=1}^{N}\alpha_{kn}k(x_{j},x_{n})+\frac{1}{|C_{k}|^{2}}\sum_{p}\sum_{q}\alpha_{kp}\alpha_{kq}k(x_{p},x_{q})$

In [6]:
def compute_cluster(K, kernel, cluster):
    new_cluster = np.zeros(10000, dtype=int)
    C = compute_C(K, cluster)
    summation_pq = compute_summation_pq(K, kernel, cluster)
    for j in range(10000):
        distance = np.zeros(K)
        for k in range(K):
            distance[k] = kernel[j, j] - (2/C[k])*(np.sum(kernel[j, :][np.where(cluster == k)])) + (1/(C[k])**2)*summation_pq[k]
        new_cluster[j] = np.argmin(distance)
    
    return new_cluster

In [7]:
def compute_C(K, cluster):
    C = np.zeros(K)
    for c in range(K):
        for n in range(10000):
            if cluster[n] == c:
                C[c] += 1
    return C

In [8]:
def compute_summation_pq(K, kernel, cluster):
    res = np.zeros(K)
    for k in range(K):
        tmp = kernel.copy()
        for n in range(len(kernel)):
            if cluster[n]!=k:
                tmp[n,:] = 0
                tmp[:,n] = 0
        res[k] = np.sum(tmp)
    return res

## Visualization

In [9]:
def save_png(cluster, image_name, iter):
    
    (filepath,tempfilename) = os.path.split(image_name)
    (filename,extension) = os.path.splitext(tempfilename)
    
    colors = np.array([[255,0,0],[0,255,0],[0,0,255],[0,215,175],[95,0,135],[255,255,0],[255,175,0]])
    result = np.zeros((100*100, 3))
    for n in range(10000):
        result[n,:] = colors[cluster[n],:]

    img = result.reshape(100, 100, 3)
    img = Image.fromarray(np.uint8(img))
    img.save(os.path.join('output', filename+f'_iter{iter}.png'))

In [10]:
def generate_gif(image_name):
    image=[]
    (filepath,tempfilename) = os.path.split(image_name)
    (filename,extension) = os.path.splitext(tempfilename)
    image_list = sorted(glob.glob(os.path.join('output/', filename + '*.png')))
    
    for i, path in enumerate(image_list):
        new = Image.open(path)
        image.append(new)
    image[0].save(filename + '.gif',format='GIF', save_all=True, append_images=image[1: ],duration=500, loop=0)
    print('gif done')

## Iteration

In [11]:
def kernel_Kmeans(K, phi_x, cluster, image_name):
    iter = 0
    while True:
        print(f'iteration {iter}')
        new_cluster = compute_cluster(K, phi_x, cluster)

        if(np.linalg.norm((new_cluster-cluster), ord=2)< 1e-2):
            break
        print(f'diff = {np.linalg.norm((new_cluster-cluster), ord=2)}')
        cluster = new_cluster
        iter += 1
        save_png(cluster, image_name, iter)

## Main Function

In [12]:
image_name = ['image1.png', 'image2.png']
mode = 'kmeans++'
K = 3
for name in image_name:
    print(f'kernel k-means for {name} start, initial center by method {mode}, pick {K} points')
    image = cv2.imread('data/'+ name, cv2.IMREAD_UNCHANGED)
    info_S, info_C = extract_info(image)
    Gram_matrix = kernel(gamma_S=0.001, gamma_C=0.01, S=info_S, C=info_C) 
    center_idx = initial_center(K=K, mode=mode)
    first_cluster = initial_cluster(center=center_idx, phi_X=Gram_matrix)
    kernel_Kmeans(K=K, phi_x=Gram_matrix, cluster=first_cluster, image_name=name)
    generate_gif(name)

kernel k-means for image1.png start, initial center by method kmeans++, pick 3 points
iteration 0
diff = 33.91164991562634
iteration 1
diff = 25.748786379167466
iteration 2
diff = 22.06807649071391
iteration 3
diff = 21.748563170931547
iteration 4
diff = 20.615528128088304
iteration 5
diff = 16.217274740226856
iteration 6
diff = 11.916375287812984
iteration 7
diff = 11.704699910719626
iteration 8
diff = 11.532562594670797
iteration 9
diff = 7.810249675906654
iteration 10
diff = 6.244997998398398
iteration 11
diff = 7.0
iteration 12
diff = 7.745966692414834
iteration 13
diff = 6.928203230275509
iteration 14
diff = 5.656854249492381
iteration 15
diff = 5.291502622129181
iteration 16
diff = 4.0
iteration 17
diff = 2.8284271247461903
iteration 18
diff = 2.0
iteration 19
gif done
kernel k-means for image2.png start, initial center by method kmeans++, pick 3 points
iteration 0
diff = 43.15089802078283
iteration 1
diff = 44.090815370097204
iteration 2
diff = 20.97617696340303
iteration 3
diff